# notebook to reformat cisbp files to meme and other issues
[all based on the files in the 2018 Lambert et al review "The Human Transcription Factors"]

In [1]:
import numpy as np
import os
import pandas as pd
import sys

## variables

In [2]:
tf_info_f = "00__metadata/TF_info.txt"
motif_info_f = "00__metadata/motif_info.txt"

In [3]:
pwm_dir = "01__pwms"

## 1. import data

In [4]:
tf_info = pd.read_table(tf_info_f, sep="\t")
tf_info.head()

,Ensembl ID,HGNC symbol,DBD,Is TF?,TF assessment,Binding mode,Motif status,Final Notes,Final Comments,Interpro ID(s),...,Vaquerizas 2009 classification,CisBP considers it a TF?,TFCat classification,Is a GO TF?,Initial assessment,Curator 1,Curator 2,TFclass considers it a TF?,Go Evidence,Pfam Domains (By ENSP ID)
0,ENSG00000137203,TFAP2A,AP-2,Yes,Known motif,Monomer or homomultimer,High-throughput in vitro,NaN,NaN,IPR008121;IPR013854,...,a,Yes,TF Gene_DNA-Binding: sequence-specific_DNA Bin...,Yes,"1a1, Direct HQ evidence",Sam Lambert,Yimeng Yin,Yes,$#ENSG00000137203#GO:0000981#sequence-specific...,$#ENSP00000368928#ENSG00000137203#ENST00000379...
1,ENSG00000008196,TFAP2B,AP-2,Yes,Known motif,Monomer or homomultimer,High-throughput in vitro,NaN,NaN,IPR008122;IPR013854,...,a,Yes,TF Gene_DNA-Binding: sequence-specific_DNA Bin...,Yes,"1a1, Direct HQ evidence",Matt Weirauch,Yimeng Yin,Yes,$#ENSG00000008196#GO:0000981#sequence-specific...,$#ENSP00000377265#ENSG00000008196#ENST00000393...
2,ENSG00000087510,TFAP2C,AP-2,Yes,Known motif,Monomer or homomultimer,High-throughput in vitro,NaN,NaN,IPR008123;IPR013854,...,a,Yes,No,Yes,"1a1, Direct HQ evidence",Matt Weirauch,Yimeng Yin,Yes,$#ENSG00000087510#GO:0001077#RNA polymerase II...,$#ENSP00000201031#ENSG00000087510#ENST00000201...
3,ENSG00000008197,TFAP2D,AP-2,Yes,Known motif,Monomer or homomultimer,In vivo/Misc source,Only known motifs are from Transfac or HocoMoc...,Binds the same GCCTGAGGC sequence as the other...,IPR013854;,...,a,Yes,No,Yes,"2a1, Lower confidence direct evidence",Arttu Jolma,Sam Lambert,Yes,$#ENSG00000008197#GO:0000981#sequence-specific...,$#ENSP00000008391#ENSG00000008197#ENST00000008...
4,ENSG00000116819,TFAP2E,AP-2,Yes,Known motif,Monomer or homomultimer,High-throughput in vitro,NaN,NaN,IPR013854;,...,a,Yes,TF Gene_DNA-Binding: sequence-specific_DNA Bin...,Yes,"1a1, Direct HQ evidence",Sam Lambert,Laura Campitelli,Yes,$#ENSG00000116819#GO:0000981#sequence-specific...,$#ENSP00000362332#ENSG00000116819#ENST00000373...


In [5]:
motif_info = pd.read_table(motif_info_f, sep="\t")
motif_info = motif_info[~pd.isnull(motif_info['CIS-BP ID'])]
motif_info.head()

,Ensembl ID,HGNC symbol,Motif evidence,Motif ID,Motif type,Motif source,CIS-BP ID,Best Motif(s)? (Figure 2A)
0,ENSG00000267281,AC023509.3,"Inferred - ATF7 (100% AA Identity, Homo sapiens)",ATF7_eDBD_HT-SELEX,HT-SELEX,Yin2017,M08490_1.94d,True
1,ENSG00000267281,AC023509.3,"Inferred - ATF7 (100% AA Identity, Homo sapiens)",ATF7_eDBD_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M08491_1.94d,NaN
2,ENSG00000267281,AC023509.3,"Inferred - Atf7 (98% AA Identity, Mus musculus)",3T3L1-Atf7_GSE56872,Misc,HOMER,M06975_1.94d,NaN
3,ENSG00000264668,AC138696.1,"Inferred - ZFP41 (100% AA Identity, Homo sapiens)",ZFP41_FL_HT-SELEX,HT-SELEX,Yin2017,M09315_1.94d,True
4,ENSG00000264668,AC138696.1,"Inferred - ZFP41 (100% AA Identity, Homo sapiens)",ZFP41_FL_Methyl-HT-SELEX,Methyl-HT-SELEX,Yin2017,M09316_1.94d,NaN


## 2. read in motif files

In [6]:
motifs = {}

In [7]:
files = os.listdir(pwm_dir)
files = [f for f in files if "README" not in f]
print("n files: %s" % (len(files)))
for f in files:
    
    motif = f.split(".")[0]
    with open("%s/%s" % (pwm_dir, f)) as fp:
        for line in fp:
            if line.startswith("Pos"):
                continue
            info = line.split()
            if info[0] == "1":
                pwm = []

            info = line.split()
            
            # round the pwm info to 5 decimal points
            info = [round(float(x), 5) for x in info]
            
            pwm.append(info[1:])
    motifs[motif] = pwm

n files: 5597


In [8]:
list(motifs.keys())[0:5]

['HKR1', 'M00117_1', 'M00118_1', 'M00119_1', 'M00120_1']

In [9]:
motifs['HKR1']

[[0.26377, 0.36123, 0.22564, 0.14936],
 [0.08157, 0.46716, 0.1197, 0.33157],
 [0.10275, 0.32309, 0.24258, 0.33157],
 [0.22987, 0.19598, 0.40784, 0.16631],
 [0.01377, 0.32733, 0.17055, 0.48835],
 [0.16208, 0.31038, 0.45869, 0.06886],
 [0.03496, 0.16208, 0.1536, 0.64936],
 [0.00106, 0.96292, 0.00106, 0.03496],
 [0.00106, 0.99259, 0.00106, 0.0053],
 [0.00106, 0.00106, 0.00106, 0.99682],
 [0.00106, 0.82733, 0.00953, 0.16208],
 [0.00106, 0.99682, 0.00106, 0.00106],
 [0.19174, 0.57309, 0.00106, 0.23411],
 [0.09004, 0.32733, 0.09428, 0.48835],
 [0.29343, 0.24258, 0.30614, 0.15784],
 [0.06886, 0.20021, 0.53919, 0.19174],
 [0.01801, 0.31886, 0.08157, 0.58157],
 [0.08581, 0.46716, 0.30614, 0.14089],
 [0.13242, 0.62818, 0.11123, 0.12818],
 [0.34004, 0.38242, 0.17055, 0.10699],
 [0.07733, 0.36547, 0.36123, 0.19598]]

## 3. map motifs to curated TFs

In [10]:
curated_tfs = set(tf_info[tf_info["Is TF?"] == "Yes"]["Ensembl ID"])
len(curated_tfs)

1639

In [11]:
curated_motif_map = {}
curated_pwms = {}

for key in motifs:
    gene = motif_info[motif_info["CIS-BP ID"].str.contains(key)]["Ensembl ID"].iloc[0]
    gene_name = motif_info[motif_info["CIS-BP ID"].str.contains(key)]["HGNC symbol"].iloc[0]
    if gene in curated_tfs:
        pwm = motifs[key]
        
        # make sure the pwm sums to 1 in all rows!
        arr = np.asarray(pwm, dtype=np.float64)
        s = arr.sum(axis=1)
        if (s < 0.99).any() or (s > 1.01).any():
            print("bad motif: %s | len: %s | sums: %s" % (key, len(pwm), s))
        else:
            curated_pwms[key] = motifs[key]
            curated_motif_map[key] = {"gene_id": gene, "gene_name": gene_name}

bad motif: M06403_1 | len: 22 | sums: [ 1.       1.       1.00001  1.       0.99999  1.       1.       1.00001
  1.       1.00001  1.       0.       0.99999  0.99999  0.99999  0.99999
  1.       1.       1.       1.       0.99999  1.     ]
bad motif: M07090_1 | len: 32 | sums: [ 1.00001  1.       1.00001  1.       1.       1.       1.00001  1.
  1.00001  1.       0.       0.99999  0.99999  1.       1.       1.       1.
  1.       1.       1.       1.       0.       1.       1.       1.00001
  0.99999  0.99999  0.99999  1.       1.       1.       1.     ]
bad motif: M07127_1 | len: 50 | sums: [ 1.       1.00001  1.       1.       0.99999  1.       0.99999  1.00001
  0.99999  1.       1.00001  1.00001  1.       1.       1.       0.       1.
  1.       1.       1.       1.       1.       1.       1.       1.       1.
  1.       1.       1.00001  1.       1.       1.       1.       1.       0.
  1.       1.00001  1.       1.       1.       1.       1.       0.99999
  1.       1.       1.00

In [12]:
curated_motif_map = pd.DataFrame.from_dict(curated_motif_map, orient="index").reset_index()
curated_motif_map.head()

,index,gene_id,gene_name
0,HKR1,ENSG00000181666,HKR1
1,M00117_1,ENSG00000137203,TFAP2A
2,M00118_1,ENSG00000137203,TFAP2A
3,M00119_1,ENSG00000137203,TFAP2A
4,M00120_1,ENSG00000137203,TFAP2A


In [13]:
len(curated_motif_map["gene_id"].unique())

927

In [14]:
len(curated_motif_map)

5586

In [15]:
curated_motif_map_f = "00__metadata/curated_motif_map.txt"
curated_motif_map.to_csv(curated_motif_map_f, sep="\t", index=False)

## 4. convert to MEME format (for FIMO)

In [16]:
out_f = "../01__meme_files/human_curated_tfs.txt"

with open(out_f, "w") as f:
    f.write("MEME version 4\n\n")
    f.write("ALPHABET= ACGT\n\n")
    f.write("strands: + -\n\n")
    f.write("Background letter frequencies (from uniform background):\nA 0.25000 C 0.25000 G 0.25000 T 0.25000\n\n")

    # now write the motifs
    for key in curated_pwms:
        #print(key)

        # first find its name
        motif_name = curated_motif_map[curated_motif_map["index"] == key]["gene_name"].iloc[0]
        f.write("MOTIF %s %s\n\n" % (key, motif_name))

        pwm = curated_pwms[key]
        n_bp = len(pwm)
        f.write("letter-probability matrix: alength= 4 w= %s\n" % n_bp)
        for pos in pwm:
            f.write("  %s\t%s\t%s\t%s\n" % (round(float(pos[0]), 5), round(float(pos[1]), 5), 
                                            round(float(pos[2]), 5), round(float(pos[3]), 5)))
        f.write("\n")
f.close()